In [1]:
import ccxt
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import pytz

# Cấu hình Binance API
binance = ccxt.binance()

# Đường dẫn file CSV
file_path = 'bitcoin_data.csv'

# Hàm lấy dữ liệu mới từ Binance
def fetch_binance_data(since=None):
    print("Đang lấy dữ liệu từ Binance...")
    # Lấy dữ liệu OHLCV mỗi phút
    ohlcv = binance.fetch_ohlcv('BTC/USDT', timeframe='1m', since=since)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    # Chuyển đổi timestamp thành datetime theo múi giờ GMT+7
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data['timestamp'] = data['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
    print(f"Đã lấy được {len(data)} dòng dữ liệu.")
    return data

# Hàm lưu dữ liệu vào CSV
def save_to_csv(data):
    print("Đang lưu dữ liệu vào file CSV...")
    # Nếu file đã tồn tại, chỉ ghi những dòng mới không trùng lặp
    if os.path.isfile(file_path):
        existing_data = pd.read_csv(file_path, parse_dates=['timestamp'])
        # Loại bỏ các dòng trùng lặp
        combined_data = pd.concat([existing_data, data]).drop_duplicates(subset='timestamp').sort_values('timestamp')
    else:
        combined_data = data
    combined_data.to_csv(file_path, index=False)
    print("Dữ liệu đã được lưu thành công.")

# Hàm bổ sung dữ liệu còn thiếu khi mở lại chương trình
def append_missing_data():
    if os.path.isfile(file_path):
        print("Đang kiểm tra dữ liệu còn thiếu...")
        existing_data = pd.read_csv(file_path, parse_dates=['timestamp'])
        last_timestamp = existing_data['timestamp'].max()
        last_timestamp = pd.to_datetime(last_timestamp)

        # Lấy dữ liệu từ thời điểm cuối cùng đến hiện tại, làm tròn phút
        start = last_timestamp + timedelta(minutes=1)
        end = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).replace(second=0, microsecond=0)

        while start < end:
            since = int(start.timestamp() * 1000)
            data = fetch_binance_data(since=since)
            if data.empty:
                break
            data = data[data['timestamp'] > last_timestamp]
            save_to_csv(data)
            last_timestamp = data['timestamp'].max()
            start = last_timestamp + timedelta(minutes=1)
    else:
        print("Không tìm thấy file dữ liệu cũ, bắt đầu tạo mới...")

# Chương trình chính
def main():
    append_missing_data()
    while True:
        # Lấy dữ liệu và lưu vào CSV
        now = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).replace(second=0, microsecond=0)
        since = int(now.timestamp() * 1000)
        data = fetch_binance_data(since=since)
        save_to_csv(data)
        
        # Đợi đến phút tiếp theo
        sleep_time = 60 - now.second - now.microsecond / 1_000_000
        print(f"Chờ {sleep_time} giây đến lần lấy dữ liệu tiếp theo...")
        time.sleep(sleep_time)

if __name__ == '__main__':
    main()

Đang kiểm tra dữ liệu còn thiếu...
Đang lấy dữ liệu từ Binance...
Đã lấy được 4 dòng dữ liệu.
Đang lưu dữ liệu vào file CSV...
Dữ liệu đã được lưu thành công.
Đang lấy dữ liệu từ Binance...
Đã lấy được 1 dòng dữ liệu.
Đang lưu dữ liệu vào file CSV...


KeyboardInterrupt: 

In [ ]:
    # Lấy dữ liệu OHLCV mỗi phút
    ohlcv = binance.fetch_ohlcv('BTC/USDT', timeframe='1m', since=since)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    # Chuyển đổi timestamp thành datetime theo múi giờ GMT+7
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data['timestamp'] = data['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
    print(f"Đã lấy được {len(data)} dòng dữ liệu.")
    return data

# Hàm lưu dữ liệu vào CSV
def save_to_csv(data):
    print("Đang lưu dữ liệu vào file CSV...")
    # Nếu file đã tồn tại, chỉ ghi những dòng mới không trùng lặp
    if os.path.isfile(file_path):
        existing_data = pd.read_csv(file_path, parse_dates=['timestamp'])
        # Loại bỏ các dòng trùng lặp
        combined_data = pd.concat([existing_data, data]).drop_duplicates(subset='timestamp').sort_values('timestamp')
    else:
        combined_data = data
    combined_data.to_csv(file_path, index=False)
    print("Dữ liệu đã được lưu thành công.")

# Hàm bổ sung dữ liệu còn thiếu khi mở lại chương trình
def append_missing_data():
    if os.path.isfile(file_path):
        print("Đang kiểm tra dữ liệu còn thiếu...")
        existing_data = pd.read_csv(file_path, parse_dates=['timestamp'])
        last_timestamp = existing_data['timestamp'].max()
        last_timestamp = pd.to_datetime(last_timestamp)

        # Lấy dữ liệu từ thời điểm cuối cùng đến hiện tại, làm tròn phút
        start = last_timestamp + timedelta(minutes=1)
        end = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).replace(second=0, microsecond=0)

        while start < end:
            since = int(start.timestamp() * 1000)
            data = fetch_binance_data(since=since)
            if data.empty:
                break
            data = data[data['timestamp'] > last_timestamp]
            save_to_csv(data)
            last_timestamp = data['timestamp'].max()
            start = last_timestamp + timedelta(minutes=1)
    else:
        print("Không tìm thấy file dữ liệu cũ, bắt đầu tạo mới...")

# Chương trình chính
def main():
    append_missing_data()
    while True:
        # Lấy dữ liệu và lưu vào CSV
        now = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).replace(second=0, microsecond=0)
        since = int(now.timestamp() * 1000)
        data = fetch_binance_data(since=since)
        save_to_csv(data)
        
        # Đợi đến phút tiếp theo
        sleep_time = 60 - now.second - now.microsecond / 1_000_000
        print(f"Chờ {sleep_time} giây đến lần lấy dữ liệu tiếp theo...")
        time.sleep(sleep_time)

if __name__ == '__main__':
    main()